# Predict an unlabeled raster and export the results

In [ ]:
import yaml
import torch
import os
import artus.inference as tusinf
import artus.spatialize as tuspal
import fiftyone as fo
import fiftyone.utils.coco as fouc

In [ ]:
raster_or_tiles_path = '/path/to/raster/or/tiles/directory/' #if your raster exceeds the cache memory then you should tile it first
config_path = '../../configs/x101_allsites_species_overlapping25_tiles5000_ITER3000.yml'

In [ ]:
dataset = tusinf.deploy_unlabeled_dataset.create_or_load_dataset(
    dataset_name=os.path.basename(config_path), #add a name for your fiftyone dataset 
    dataset_type='unlabeled', 
    images_path='raster_or_tiles_path',
    label_type='segmentations')

dataset.persistent = True #optional : save dataset on your machine for further exploration

dataset.save()

print(dataset)

## Call the trained model

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

#Load model's classes
model_classes = '../../configs/model_classes_species.yml'

with open(model_classes) as f:
    model_classes = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
predictor = tusinf.predict.build_predictor(config_path, device)

## Make predictions on images with the AI model

In [ ]:
dataset = tusinf.predict.add_predictions_to_dataset(
    dataset=dataset, 
    predictor=predictor, 
    device=device, 
    classes=model_classes['species_classes'], #the list of a class names or config file containing the list
    predictions_field='predictions', 
    nms_threshold=0.5)

## Spatialize model's predictions and export them

In [ ]:
coco_exporter = fouc.COCODetectionDatasetExporter(
    export_dir='/path/to/export/dir/',
    labels_path= 'coco.json',
    export_media=False,
    classes=model_classes['species_classes'],
    tolerance=1, 
    extra_attrs=False)

dataset.export(
    dataset_exporter=coco_exporter,
    label_field='predictions')

In [ ]:
geojson_exporter = tuspal.GeoCOCOExporter(
    coco_path = 'path/to/coco/file.json', #the path you indicated in the last chunk
    sample_dir = 'path/to/tif/directory/',
    epsg_code = 'EPSG:4326', #set the epsg adapted to your data
    dest_path = 'path/to/export/directory/',
    dest_name = 'predictions.geojson'
)

In [ ]:
geojson_exporter.export()